In [5]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Logical;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

In [6]:
// Task 1 (1 point). f(x) = 1 if x is divisible by 4
//         
// Inputs:
//      1) an N-qubit array "inputs" (3 ≤ N ≤ 5),
//      2) a qubit "output".
// Goal: Implement a marking oracle for function f(x) = 1 if x is divisible by 4.
//       That is, if both inputs are in a basis state, flip the state of the output qubit 
//       if and only if the number written in the array "inputs" is divisible by 4,
//       and leave the state of the array "inputs" unchanged.
//       The array "inputs" uses little-endian encoding, i.e., 
//       the least significant bit of the integer is stored first.
//       The effect of the oracle on superposition states should be defined by its linearity.
//       Don't use measurements; the implementation should use only X gates and its controlled variants.
//       This task will be tested using ToffoliSimulator.
// 
// Example: the result of applying the oracle to a state (|001⟩ + |100⟩ + |111⟩)/√3 ⊗ |0⟩
// will be 1/√3|001⟩ ⊗ |1⟩ + 1/√3|100⟩ ⊗ |0⟩ + 1/√3|111⟩ ⊗ |0⟩.
//

In [22]:
operation Task1_DivisibleByFour (inputs : Qubit[], output : Qubit) : Unit is Adj+Ctl
{
    ApplyToEachCA(X, inputs);
    Controlled X(inputs[0..2], output);
}

Task1_DivisibleByFour

In [23]:
operation VerifySingleOutputFunction(numInputs : Int, op : ((Qubit[], Qubit) => Unit is Adj+Ctl), predicate : (Int -> Bool)) : Unit {
        for assignment in 0 .. 2^numInputs - 1 {
            use (inputs, output) = (Qubit[numInputs], Qubit());
            within {
                ApplyXorInPlace(assignment, LittleEndian(inputs));
                AllowAtMostNCallsCA(0, Measure, "You are not allowed to use measurements");
            } apply {
                op(inputs, output);
            }

            // Check that the result is expected
            let actual = ResultAsBool(MResetZ(output));
            let expected = predicate(assignment);
            Fact(actual == expected,
                $"Oracle evaluation result {actual} does not match expected {expected} for assignment {IntAsBoolArray(assignment, numInputs)}");

            // Check that the inputs were not modified
            Fact(MeasureInteger(LittleEndian(inputs)) == 0, 
                $"The input states were modified for assignment {assignment}");
        }
    }


    // ------------------------------------------------------
    function IsDivisibleByFour (input : Int) : Bool {
        return input % 4 == 0;
    }
    
    operation Task1_DivisibleByFour (inputs : Qubit[], output : Qubit) : Unit is Adj+Ctl
    {
        ApplyToEachCA(X, inputs);
        Controlled X(inputs[0..2], output);
    }

    
    operation Test1_DivisibleByFour() : Unit {
        for i in 3 .. 5 {
            VerifySingleOutputFunction(3, Task1_DivisibleByFour, IsDivisibleByFour);
        }
    }

VerifySingleOutputFunction, IsDivisibleByFour, Task1_DivisibleByFour, Test1_DivisibleByFour

In [24]:
%simulate Test1_DivisibleByFour

Source,Callable
(notebook),VerifySingleOutputFunction
(notebook),Test1_DivisibleByFour


The input states were modified for assignment 0
